In [12]:
import json
import os
import shutil
import numpy as np

PATH_DATASET_1 = './Data/Data_1/1_AeULDs'
PATH_DATASET_2 = './Data/Data_2/1_AeULDs'

INPUT_PATH = PATH_DATASET_2

#Defines, in which area the com will be located and the distribution will be bounded (along item's dimensions)
COM_BOUNDARIES = 0.8
INPUT_PATH_ULDS_SCENARIO_1 = INPUT_PATH+"/Ulds_scenario_1/"
OUTPUT_PATH_NORMAL = INPUT_PATH+"/Ulds_scenario_2a/"
OUTPUT_PATH_UNIFORM = INPUT_PATH+"/Ulds_scenario_2b/"

RANDOM_SEED = 1
np.random.seed(RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)

In [13]:
def readJson(filePathImport):
    #Read file in
    with open(filePathImport) as json_file:
        job_json = json.load(json_file)
        return job_json

In [14]:
def writeJson(filePathExport, jsonFile):
    with open(filePathExport, 'w', encoding='utf-8') as f:
        json.dump(jsonFile, f, ensure_ascii=False, indent=4)

In [15]:
def createOutputFolderIfNotExists(path):
    isExist = os.path.exists(path)
    if isExist:
        shutil.rmtree(path)
        os.makedirs(path)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(path)
       print("The new directory is created!")

In [16]:
def transformSingleULD(fileName, mode):
    try:
        uld = readJson(INPUT_PATH_ULDS_SCENARIO_1 + fileName)
    except:
        print("Could not read file! " + fileName)
        return  
    
    for item in uld["placedItems"]:
        item["centerOfMass"] = createCOM(item, mode)
       
    if mode == 'NORMAL':
        writeJson(OUTPUT_PATH_NORMAL + fileName, uld)
    elif mode == 'UNIFORM':
        writeJson(OUTPUT_PATH_UNIFORM + fileName, uld)

In [17]:
def createCOM(item, mode):
    if item['shape']['shapeType'] != 'Box':
        print("No Box provided!")
        return
    
    if mode == 'NORMAL':
        x = getNormalDistributedNumerInBound(item['shape']['width'])
        z = getNormalDistributedNumerInBound(item['shape']['depth'])
        y = getNormalDistributedNumerInBound(item['shape']['height'])
    elif mode == 'UNIFORM':
        x = getUniformDistributedNumerInBound(item['shape']['width'])
        z = getUniformDistributedNumerInBound(item['shape']['depth'])
        y = getUniformDistributedNumerInBound(item['shape']['height'])        
    
    com = {
        'x': x,
        'y': y,
        'z': z
    }
    
    return com

In [18]:
def getNormalDistributedNumerInBound(upperBound):
    upperBound = upperBound * COM_BOUNDARIES
    
    sigma =  upperBound / 6
    n = np.random.normal(loc=0.0, scale=sigma)
    if n < -(upperBound / 2):
        return -(upperBound / 2)
    
    if n > upperBound / 2:
        return upperBound / 2
    
    return n

In [19]:
def getUniformDistributedNumerInBound(upperBound):
    # clip bounds at 80 %
    upperBound = upperBound * COM_BOUNDARIES
    n = np.random.uniform(low = -(upperBound / 2), high = (upperBound / 2))
    return n

In [20]:
from tqdm import tqdm

def transformULDs(scenario):
    for f in tqdm(os.listdir(INPUT_PATH_ULDS_SCENARIO_1)):
        transformSingleULD(f, scenario)

In [21]:
createOutputFolderIfNotExists(OUTPUT_PATH_NORMAL)
transformULDs('NORMAL')

The new directory is created!


100%|██████████| 9689/9689 [00:13<00:00, 708.72it/s]


In [22]:
createOutputFolderIfNotExists(OUTPUT_PATH_UNIFORM)
transformULDs('UNIFORM')

The new directory is created!


100%|██████████| 9689/9689 [00:14<00:00, 680.72it/s]
